In [2]:
import pandas as pd
import numpy as np
import pickle
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
nltk.download('wordnet')
from text_cleaning import lemmatize_and_stem, preprocess, get_aggregate_score
import time
import urllib.request
from collections import Counter
from selenium.webdriver import Chrome

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kaylischulz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# countries_df = pickle.load(open('data/countries.pkl', 'rb'))
# countries_df.head()

,country,country_summary
0,Austria,"Small, landlocked Austria offers alpine scener..."
1,Belgium,Belgium falls through the cracks. Wedged betwe...
2,Bosnia-Herzegovina,Apart from the tragic way it separated from Yu...
3,Bulgaria,"Endearing, surprising Bulgaria is a rewarding ..."
4,Croatia,With thousands of miles of seafront and more t...


In [4]:
# cities_df = pickle.load(open('data/cities.pkl', 'rb'))
# cities_df.head()

,city,city_summary,country
0,Danube Valley,The Danube is at its romantic best just west o...,Austria
1,Hallstatt,Lovable Hallstatt is a tiny town bullied onto ...,Austria
2,Salzburg,"Thanks to its charmingly preserved old town, s...",Austria
3,Tirol,Mountainous Tirol — in Austria's western panha...,Austria
4,Vienna,"Vienna is the capital of Austria, the cradle o...",Austria


In [5]:
# wiki_city_df = pickle.load(open('data/wiki_data.pkl', 'rb'))

In [6]:
# wiki_city_df.head()

,_id,city,text
0,5d1195dc0d668bb1db943136,Vienna,\nVienna (/viˈɛnə/ (listen);[11][12] German: W...
1,5d11970b6e7463927710e427,Vienna,\nVienna (/viˈɛnə/ (listen);[11][12] German: W...
2,5d11974d6e7463927710e428,Danube Valley,\nThe Danube (/ˈdæn.juːb/ DAN-yoob; known by v...
3,5d1197626e7463927710e429,Danube Valley,\nThe Danube (/ˈdæn.juːb/ DAN-yoob; known by v...
4,5d1197936e7463927710e42a,Hallstatt,Hallstatt (German: [ˈhalʃtat]; Central Bavaria...


In [7]:
# wiki_city_df.drop('_id', axis=1, inplace=True)
# wiki_city_df.head()

,city,text
0,Vienna,\nVienna (/viˈɛnə/ (listen);[11][12] German: W...
1,Vienna,\nVienna (/viˈɛnə/ (listen);[11][12] German: W...
2,Danube Valley,\nThe Danube (/ˈdæn.juːb/ DAN-yoob; known by v...
3,Danube Valley,\nThe Danube (/ˈdæn.juːb/ DAN-yoob; known by v...
4,Hallstatt,Hallstatt (German: [ˈhalʃtat]; Central Bavaria...


In [8]:
# wiki_city_df.drop_duplicates(inplace=True)

In [9]:
# wiki_city_df.head()

,city,text
0,Vienna,\nVienna (/viˈɛnə/ (listen);[11][12] German: W...
2,Danube Valley,\nThe Danube (/ˈdæn.juːb/ DAN-yoob; known by v...
4,Hallstatt,Hallstatt (German: [ˈhalʃtat]; Central Bavaria...
5,Salzburg,Salzburg (German: [ˈzaltsbʊɐ̯k] (listen);[note...
6,Tirol,"\nTyrol (/tɪˈroʊl, taɪ-, ˈtaɪroʊl/;[1] histori..."


In [14]:
# for row in wiki_city_df['text']:
#     print(row[:300])
#     print('\n')


Vienna (/viˈɛnə/ (listen);[11][12] German: Wien [viːn] (listen)) is the federal capital, largest city and one of nine states of Austria. Vienna is Austria's primate city, with a population of about 1.9 million[3] (2.6 million within the metropolitan area,[6] nearly one third of the country's popula



The Danube (/ˈdæn.juːb/ DAN-yoob; known by various names in other languages) is Europe's second longest river, after the Volga. It is located in Central and Eastern Europe.
The Danube was once a long-standing frontier of the Roman Empire, and today flows through 10 countries, more than any other ri


Hallstatt (German: [ˈhalʃtat]; Central Bavarian: Hoistod) is a small town in the district of Gmunden, in the Austrian state of Upper Austria. Situated between the southwestern shore of Hallstätter See and the steep slopes of the Dachstein massif, the town lies in Salzkammergut region, on the nationa


Salzburg (German: [ˈzaltsbʊɐ̯k] (listen);[note 1] literally "Salt Fortress") is the capital

Since the data set is small enough and it is possible that wikipedia sent to a different link than intended (due to ambiguity), I will do a visual check that the summary is the summary I want.

Cities I need to recollect:
* Danube Valley, Austria (references the river, I want to get the valley (city such as Melk))
* Split, Croatia
* Bath, England
* Durham, Englang
* Glastonbury & Wells, England (may need to get for each city?)
* Stonehenge & Avebury, England (may need to refer to each city/landmark)
* Stratford, England
* Warwick & Coventry, England
* Windsor, England
* D-Day Beaches, France
* Reims & Verdun, France (may need to get for each city)
* Rhine Valley, Germany
* Hydra, Greece
* Olympia, Greece
* Connemara & County Mayo, Ireland
* Country Clare & the Burren, Ireland
* Kenmare & the Ring of Kerry, Ireland
* Kilkenny & the Rock of Cashel, Ireland
* Kinsale & Cobh, Ireland
* Portrush & the Antrim Coast, Ireland
* Waterford & County Wexford, Ireland
* Italian Lakes, Italy
* Pompeii & Herculaneum, Italy
* Tuscan Hill Towns, Italy
* Edam, Netherlands
* Norwegian Fjords, Norway
* Nazaré, Portugal
* Óbidos, Portugal
* Oban, Mull & Iona, Scotland
* Córdoba, Spain
* Toledo, Spain
* White Hill Towns, Spain
* Lake Geneva & French Switzerland, Switzerland


In [14]:
# browser = Chrome()

In [12]:
# city_wiki = [('Danube Valley', 'Melk'),
#              ('Split', 'Split,_Croatia'), ('Bath', 'Bath,_Somerset'), ('Durham', 'Durham,_England'), 
#              ('Glastonbury & Wells', 'Glastonbury'), ('Stonehenge & Avebury', 'Avebury'),
#              ('Stratford', 'Stratford-upon-Avon'), ('Warwick & Coventry', 'Coventry'),
#              ('Windsor', 'Windsor,_Berkshire'), ('D-Day Beaches', 'Normandy_landings'),
#              ('Reims & Verdun', 'Reims'), ('Rhine Valley', 'Mörsbach'), ('Hydra', 'Hydra_(island)'),
#              ('Olympia', 'Olympia,_Greece'), ('Connemara & County Mayo', 'Connemara'),
#              ('Country Clare & the Burren', 'The_Burren'), ('Kenmare & the Ring of Kerry', 'Kenmare'),
#              ('Kilkenny & the Rock of Cashel', 'Kilkenny'), ('Kinsale & Cobh', 'Cobh'),
#              ('Portrush & the Antrim Coast', 'Portrush'), ('Waterford & County Wexford', 'Waterford'),
#              ('Italian Lakes', 'Varenna'), ('Pompeii & Herculaneum', 'Pompeii'),
#              ('Tuscan Hill Towns', 'Montepulciano'), ('Edam', 'Edam,_Netherlands'),
#              ('Norwegian Fjords', 'Sognefjord'), ('Nazaré', 'Nazaré,_Portugal'),
#              ('Óbidos', 'Óbidos,_Portugal'), ('Oban, Mull & Iona', 'Iona'), ('Córdoba', 'Córdoba,_Spain'),
#              ('Toledo', 'Toledo,_Spain'), ('White Hill Towns', 'Ronda'),
#              ('Lake Geneva & French Switzerland', 'Lake_Geneva'),
#             ]

In [15]:
# for city in city_wiki:
#     replace_df_text(browser, city, wiki_city_df)
#     time.sleep(np.random.randint(10,30))
#     print(f'Replaced {city[0]}')

Replaced Danube Valley
Replaced Split
Replaced Bath
Replaced Durham
Replaced Glastonbury & Wells
Replaced Stonehenge & Avebury
Replaced Stratford
Replaced Warwick & Coventry
Replaced Windsor
Replaced D-Day Beaches
Replaced Reims & Verdun
Replaced Rhine Valley
Replaced Hydra
Replaced Olympia
Replaced Connemara & County Mayo
Replaced Country Clare & the Burren
Replaced Kenmare & the Ring of Kerry
Replaced Kilkenny & the Rock of Cashel
Replaced Kinsale & Cobh
Replaced Portrush & the Antrim Coast
Replaced Waterford & County Wexford
Replaced Italian Lakes
Replaced Pompeii & Herculaneum
Replaced Tuscan Hill Towns
Replaced Edam
Replaced Norwegian Fjords
Replaced Nazaré
Replaced Óbidos
Replaced Oban, Mull & Iona
Replaced Córdoba
Replaced Toledo
Replaced White Hill Towns
Replaced Lake Geneva & French Switzerland


In [10]:
# combined_descriptions_df = pd.merge(cities_df, wiki_city_df, how='left', on='city')
# combined_descriptions_df.head(10)

,city,city_summary,country,_id,text
0,Danube Valley,The Danube is at its romantic best just west o...,Austria,5d11974d6e7463927710e428,\nThe Danube (/ˈdæn.juːb/ DAN-yoob; known by v...
1,Danube Valley,The Danube is at its romantic best just west o...,Austria,5d1197626e7463927710e429,\nThe Danube (/ˈdæn.juːb/ DAN-yoob; known by v...
2,Hallstatt,Lovable Hallstatt is a tiny town bullied onto ...,Austria,5d1197936e7463927710e42a,Hallstatt (German: [ˈhalʃtat]; Central Bavaria...
3,Salzburg,"Thanks to its charmingly preserved old town, s...",Austria,5d1197ff6e7463927710e42b,Salzburg (German: [ˈzaltsbʊɐ̯k] (listen);[note...
4,Tirol,Mountainous Tirol — in Austria's western panha...,Austria,5d1198416e7463927710e42c,"\nTyrol (/tɪˈroʊl, taɪ-, ˈtaɪroʊl/;[1] histori..."
5,Vienna,"Vienna is the capital of Austria, the cradle o...",Austria,5d1195dc0d668bb1db943136,\nVienna (/viˈɛnə/ (listen);[11][12] German: W...
6,Vienna,"Vienna is the capital of Austria, the cradle o...",Austria,5d11970b6e7463927710e427,\nVienna (/viˈɛnə/ (listen);[11][12] German: W...
7,Vienna,"Vienna is the capital of Austria, the cradle o...",Austria,5d11988a6e7463927710e42d,\nVienna (/viˈɛnə/ (listen);[11][12] German: W...
8,Antwerp,"Antwerp (Antwerpen in Dutch, Anvers in French)...",Belgium,5d1198cd6e7463927710e42e,\nAntwerp (/ˈæntwɜːrp/ (listen); Dutch: Antwer...
9,Bruges,"With pointy gilded architecture, stay-a-while ...",Belgium,5d1198ef6e7463927710e42f,"\nBruges (/bruːʒ/, French: [bʁyʒ]; Flemish: Br..."


In [11]:
#pickle.dump(combined_descriptions_df, open('data/combined_cities.pkl', 'wb'))
combined_descriptions_df = pickle.load(open('data/combined_cities.pkl', 'rb'))

In [12]:
processed_countries = list(combined_descriptions_df['city_summary'].map(preprocess))
processed_countries.extend(list(combined_descriptions_df['text'].map(preprocess)))

In [13]:
len(processed_countries)

426

In [15]:
dictionary_bow = gensim.corpora.Dictionary(processed_countries)

In [16]:
dictionary_bow.filter_extremes(no_below=5, no_above=0.30)

In [17]:
bow_corpus = [dictionary_bow.doc2bow(country) for country in processed_countries]

In [95]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5,
                                       id2word=dictionary_bow, passes=20,
                                       workers=1, alpha=20)

for ind, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(ind, topic))

Topic: 0 
Words: 0.009*"forest" + 0.007*"valley" + 0.004*"natur" + 0.004*"mountain" + 0.004*"bc" + 0.004*"rout" + 0.003*"abbey" + 0.003*"monument" + 0.003*"univers" + 0.003*"landscap"
Topic: 1 
Words: 0.006*"palac" + 0.006*"saint" + 0.004*"million" + 0.004*"host" + 0.004*"univers" + 0.004*"urban" + 0.004*"council" + 0.003*"mayor" + 0.003*"metropolitan" + 0.003*"institut"
Topic: 2 
Words: 0.016*"island" + 0.005*"bc" + 0.005*"lake" + 0.004*"empir" + 0.004*"mountain" + 0.004*"rule" + 0.004*"pope" + 0.003*"di" + 0.003*"provinc" + 0.003*"averag"
Topic: 3 
Words: 0.006*"camp" + 0.005*"prison" + 0.005*"speak" + 0.005*"unit" + 0.004*"beach" + 0.004*"cathol" + 0.004*"land" + 0.004*"alli" + 0.004*"island" + 0.003*"armi"
Topic: 4 
Words: 0.008*"univers" + 0.007*"school" + 0.006*"council" + 0.006*"club" + 0.005*"theatr" + 0.005*"festiv" + 0.004*"music" + 0.004*"compani" + 0.004*"bridg" + 0.004*"colleg"


In [8]:
# pickle.dump(lda_model, open('models/lda_model.pkl', 'wb'))
lda_model = pickle.load(open('models/lda_model.pkl', 'rb'))

In [18]:
lda_model[bow_corpus[228]]

[(0, 0.07664846),
 (1, 0.08688753),
 (2, 0.5106748),
 (3, 0.24573861),
 (4, 0.08005066)]

In [19]:
aggregate_scores = []
for i in range(213):
    scores = get_aggregate_score(lda_model, bow_corpus, i)
    aggregate_scores.append(scores)
aggregate_scores

[{0: 0.24338701, 1: 0.22506629, 2: 0.16711442, 3: 0.18475968, 4: 0.17967261},
 {0: 0.33294913, 1: 0.15219468, 2: 0.19370267, 3: 0.1703875, 4: 0.150766},
 {0: 0.14682823, 1: 0.3035228, 2: 0.16091079, 3: 0.22598091, 4: 0.16275725},
 {0: 0.1620687, 1: 0.2027069, 2: 0.26222855, 3: 0.22460902, 4: 0.14838687},
 {0: 0.122002356, 1: 0.49321416, 2: 0.108736224, 3: 0.13811629, 4: 0.13793093},
 {0: 0.13308576, 1: 0.18711817, 2: 0.13865304, 3: 0.19672814, 4: 0.3444149},
 {0: 0.14376245, 1: 0.2031925, 2: 0.17108789, 3: 0.1750706, 4: 0.3068866},
 {0: 0.11628233, 1: 0.48226288, 2: 0.11501013, 3: 0.14246842, 4: 0.14397624},
 {0: 0.17076766, 1: 0.27140012, 2: 0.14306162, 3: 0.16206354, 4: 0.25270706},
 {0: 0.15473919, 1: 0.18967897, 2: 0.29929644, 3: 0.1437868, 4: 0.21249864},
 {0: 0.12120261, 1: 0.23156145, 2: 0.32904005, 3: 0.15634371, 4: 0.16185215},
 {0: 0.12807061, 1: 0.11610804, 2: 0.22618014, 3: 0.41845345, 4: 0.111187756},
 {0: 0.13572282, 1: 0.15784915, 2: 0.18662837, 3: 0.38194287, 4: 0.13785

In [20]:
topics_df = pd.DataFrame(aggregate_scores)

In [21]:
topics_df.head()

,0,1,2,3,4
0,0.243387,0.225066,0.167114,0.184760,0.179673
1,0.332949,0.152195,0.193703,0.170388,0.150766
2,0.146828,0.303523,0.160911,0.225981,0.162757
3,0.162069,0.202707,0.262229,0.224609,0.148387
4,0.122002,0.493214,0.108736,0.138116,0.137931


In [22]:
col_names=['forest_mountain','palaces','island_water', 'historical_ww2', 'urban']
topics_df.columns = col_names
topics_df.head()

,forest_mountain,palaces,island_water,historical_ww2,urban
0,0.243387,0.225066,0.167114,0.184760,0.179673
1,0.332949,0.152195,0.193703,0.170388,0.150766
2,0.146828,0.303523,0.160911,0.225981,0.162757
3,0.162069,0.202707,0.262229,0.224609,0.148387
4,0.122002,0.493214,0.108736,0.138116,0.137931


In [23]:
cities_and_topics = pd.merge(cities_df, topics_df, left_index=True, right_index=True)

In [24]:
cities_and_topics.head(10)

,city,city_summary,country,forest_mountain,palaces,island_water,historical_ww2,urban
0,Danube Valley,The Danube is at its romantic best just west o...,Austria,0.243387,0.225066,0.167114,0.184760,0.179673
1,Hallstatt,Lovable Hallstatt is a tiny town bullied onto ...,Austria,0.332949,0.152195,0.193703,0.170388,0.150766
2,Salzburg,"Thanks to its charmingly preserved old town, s...",Austria,0.146828,0.303523,0.160911,0.225981,0.162757
3,Tirol,Mountainous Tirol — in Austria's western panha...,Austria,0.162069,0.202707,0.262229,0.224609,0.148387
4,Vienna,"Vienna is the capital of Austria, the cradle o...",Austria,0.122002,0.493214,0.108736,0.138116,0.137931
5,Antwerp,"Antwerp (Antwerpen in Dutch, Anvers in French)...",Belgium,0.133086,0.187118,0.138653,0.196728,0.344415
12,Bruges,"With pointy gilded architecture, stay-a-while ...",Belgium,0.135723,0.157849,0.186628,0.381943,0.137857
13,Brussels,"Six hundred years ago, Brussels was just a nic...",Belgium,0.153482,0.120696,0.428276,0.167222,0.130324
27,Ghent,"Ghent doesn't ooze with cobbles and charm, lik...",Belgium,0.336216,0.121956,0.168517,0.119888,0.253423
28,Mostar,"Despite the scars of war, Mostar is still stun...",Bosnia-Herzegovina,0.139975,0.120362,0.133674,0.144842,0.461147


In [25]:
cities_and_topics.describe()

,forest_mountain,palaces,island_water,historical_ww2,urban
count,194.000000,194.000000,194.000000,194.000000,194.000000
mean,0.199578,0.201660,0.209817,0.178939,0.209982
std,0.089102,0.095744,0.101341,0.071911,0.110210
min,0.101850,0.094428,0.100979,0.096128,0.094359
25%,0.134442,0.140022,0.129981,0.134817,0.141130
50%,0.175854,0.170049,0.172998,0.168827,0.170200
75%,0.220561,0.231418,0.267087,0.193976,0.223130
max,0.574893,0.548769,0.570963,0.598127,0.533665


In [26]:
#pickle.dump(cities_and_topics, open('data/cities_with_topic_scores.pkl', 'wb'))